# 23.2. Computing Entropy (10 points)
- Collection of files to work with: Pizza and Chili Corpus
- Compute k-th order entropy of files in the dataset for k=0,1,2,3,4.
- Compute approximate size of the compressed file if encoded by Hk bits per symbol. Length times H_k.

In [3]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.sep.join([".."]*2))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [5]:
import os
from collections import Counter
import numpy as np
import math
import pandas as pd
import plotly.express as px
import time
from src.types.table_types import Fields
from src.load_data import get_dataset
from src.load_data import DataSets
from src.get_probs import get_sorted_probs_as_df
import numpy as np
import pandas as pd
import time
from src.types.table_types import Fields

In [6]:
def load_data(path):
    content = ''
    with open(path, 'r', encoding='windows-1252', errors='replace') as f:
        content = f.read()
    return content

In [7]:
directory = f'..\..\datasets'

In [8]:
def get_alphabet(data):
    c = dict(Counter(data))
    return list(c.keys())

In [9]:
def get_k_words_with_freq(text, k):
    res = {}

    tic = time.time()
    
    for i in range(0, len(text)-k+1):
        current_text = text[i:i+k]
        res[current_text] = res.get(current_text, 0) + 1

    toc = time.time()

    return (res, toc - tic)

In [10]:
def calc_with_zero(nom, den):
    if den == 0:
        return 0
    return nom / den

In [11]:
def get_sub_k_with_freq(data, keys, k):
    tic = time.time()

    k_1 = k + 1

    res = { k:{} for k in keys }
    
    for i in range(0, len(data) - k_1 + 1):

        current_text = data[i:i+k_1]
        
        prefix = current_text[0:k]

        current_dic = res[prefix]

        current_dic[current_text] = current_dic.get(current_text, 0) + 1

        res[prefix] = current_dic
    
    toc = time.time()

    return res, toc - tic

In [12]:
def calculate_entropy_for_state(state_k_dic, state_k_plus_1_dic):
    tic = time.time()

    res = {}

    states_n = np.sum(list(state_k_dic.values()))

    for state, state_freq in state_k_dic.items():
        suma = 0
        for state_k_plus_1, state_k_plus_1_freq in state_k_plus_1_dic[state].items():
            suma += calc_with_zero(state_k_plus_1_freq, states_n) * math.log2(state_k_plus_1_freq / state_freq) 
        res[state] = suma


    toc = time.time()
    return -np.sum(list(res.values())), toc - tic, res

In [22]:
def run(path, k, name=None):
   data = load_data(path)

   n = len(data)

   states_with_freq, k_freq_time = get_k_words_with_freq(data, k)

   k_plus_1_states_with_freq, k_plus_1_freq_time = get_sub_k_with_freq(data, list(states_with_freq.keys()), k)

   finate, calculation_time, state_entropies =  calculate_entropy_for_state(states_with_freq, k_plus_1_states_with_freq)


   return {
      "finate": finate,
      "k": k,
      "path": path,
      "aproximate_size": finate * n,
      "original_size": n * 8,
      "k_freq_time": k_freq_time, 
      "k_plus_1_freq_time": k_plus_1_freq_time,
      "calculation_time": calculation_time,
      "state_entropies": state_entropies,

      Fields.Path.value: path,
      Fields.Parameter.value: k,
      Fields.Type.value: f"{k}-Entropy",
      Fields.Original.value: n * 8,
      Fields.Aprox.value: finate * n,
      Fields.Calculation.value:  k_freq_time + k_plus_1_freq_time,
      Fields.Name.value: name
   }

In [23]:
all_directories = []


for dir in os.listdir(directory):
    for nested_dir in os.listdir(os.path.sep.join([directory, dir])):
       all_directories.append(os.path.sep.join([directory, dir, nested_dir])) 

In [24]:
all_directories[1:2]

['..\\..\\datasets\\english\\english.50MB']

In [25]:
data_dna, path_dna = get_dataset(DataSets.dna)
data_english, path_english = get_dataset(DataSets.english)
datasets = [(data_dna, path_dna, DataSets.dna.value), (data_english, path_english, DataSets.english.value)]

Loading C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB
Loading C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB


In [26]:
res = {}
counter = 0
from_k = 1
to_k = 6


for data, path, name in datasets:
    result = None
    for k in range(from_k, to_k):
        print(f'{path} - {k}')
        full_path = path
        result = run(full_path, k, name)
        res[counter] = result
        counter += 1

C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB - 1
C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB - 2
C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB - 3
C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB - 4
C://Users//proko//Desktop//University//iv//aks//datasets\dna\dna.50MB - 5
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB - 1
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB - 2
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB - 3
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB - 4
C://Users//proko//Desktop//University//iv//aks//datasets\english\english.50MB - 5


In [27]:
result_dataframe = pd.DataFrame.from_dict(res, orient="index")

In [28]:
result_dataframe

,finate,k,path,aproximate_size,original_size,k_freq_time,k_plus_1_freq_time,calculation_time,state_entropies,Path,Parameter,Type,Original,Aprox,Calculation,Name
0,1.934922,1,C://Users//proko//Desktop//University//iv//aks...,1.014456e+08,419430400,18.715345,31.584019,0.000000,"{'G': -0.4182296166786281, 'A': -0.57031832158...",C://Users//proko//Desktop//University//iv//aks...,1,1-Entropy,419430400,1.014456e+08,50.299364,dna
1,1.924513,2,C://Users//proko//Desktop//University//iv//aks...,1.008999e+08,419430400,22.820124,39.055231,0.001001,"{'GA': -0.11694824052442548, 'AT': -0.14553541...",C://Users//proko//Desktop//University//iv//aks...,2,2-Entropy,419430400,1.008999e+08,61.875355,dna
2,1.919584,3,C://Users//proko//Desktop//University//iv//aks...,1.006415e+08,419430400,25.097198,40.339319,0.002000,"{'GAT': -0.02592923929629321, 'ATC': -0.023003...",C://Users//proko//Desktop//University//iv//aks...,3,3-Entropy,419430400,1.006415e+08,65.436517,dna
3,1.912851,4,C://Users//proko//Desktop//University//iv//aks...,1.002885e+08,419430400,26.833465,38.468441,0.004003,"{'GATC': -0.004477112476998099, 'ATCA': -0.009...",C://Users//proko//Desktop//University//iv//aks...,4,4-Entropy,419430400,1.002885e+08,65.301906,dna
4,1.903402,5,C://Users//proko//Desktop//University//iv//aks...,9.979306e+07,419430400,23.166569,37.941295,0.009001,"{'GATCA': -0.0017566910040682701, 'ATCAA': -0....",C://Users//proko//Desktop//University//iv//aks...,5,5-Entropy,419430400,9.979306e+07,61.107864,dna
5,3.606487,1,C://Users//proko//Desktop//University//iv//aks...,1.890838e+08,419430400,17.445077,32.305796,0.005992,"{' ': -0.0956065483004109, 'T': -0.00488728286...",C://Users//proko//Desktop//University//iv//aks...,1,1-Entropy,419430400,1.890838e+08,49.750874,english
6,2.921758,2,C://Users//proko//Desktop//University//iv//aks...,1.531843e+08,419430400,24.371527,41.400030,0.062999,"{' ': -0.01029982260502319, ' T': -0.00054962...",C://Users//proko//Desktop//University//iv//aks...,2,2-Entropy,419430400,1.531843e+08,65.771558,english
7,2.385559,3,C://Users//proko//Desktop//University//iv//aks...,1.250720e+08,419430400,32.719821,50.555319,0.364988,"{' ': -0.001002157849502695, ' T': -0.00021...",C://Users//proko//Desktop//University//iv//aks...,3,3-Entropy,419430400,1.250720e+08,83.275140,english
8,2.012510,4,C://Users//proko//Desktop//University//iv//aks...,1.055135e+08,419430400,29.750374,69.147515,1.708000,"{' ': -0.0005175874720459757, ' T': -3.70...",C://Users//proko//Desktop//University//iv//aks...,4,4-Entropy,419430400,1.055135e+08,98.897889,english
9,1.763829,5,C://Users//proko//Desktop//University//iv//aks...,9.247546e+07,419430400,42.746000,72.720281,3.910003,"{' ': -0.0002271166314822888, ' T': -2....",C://Users//proko//Desktop//University//iv//aks...,5,5-Entropy,419430400,9.247546e+07,115.466281,english


In [29]:
from src.save import save_both

In [30]:
save_both(result_dataframe)

# Graphs

In [17]:
fig = px.line(result_dataframe, x="k", y="finate", color='path')
fig.show()